In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from pandas import Series

In [2]:
url = "http://www.omdbapi.com/?apikey=trilogy&t="

In [3]:
#read in to CSV File
movies_df = pd.read_csv("2013-2017 Box Office Data.csv")


In [4]:
#Clean up Movie Titles, removing (2015) from Cinderalla, (2016) Jungle Book, (2017) Beauty and the Beast
movies_df = movies_df.reset_index(level=None, drop = False, inplace = False, col_level = 0, col_fill = "")
s = pd.DataFrame(movies_df["Movie Title"])
s1 = s.reset_index(level=None, drop = False, inplace = False, col_level = 0, col_fill = "")
s2 = s1["Movie Title"].str.split("(").apply(Series, 1)
s3 = s2.reset_index(level=None, drop = False, inplace = False, col_level = 0, col_fill = "")

In [5]:
#merge the dataframes
movies_merge = movies_df.merge(s3, how='inner', on="index", left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False)


In [6]:
#delete excessive columns 
del movies_merge["Movie Title"]
del movies_merge[1]
del movies_merge["index"]
del movies_merge["Theaters.1"]
movies_merge = movies_merge.rename(columns = {0:"Movie Title"})
movies_df = movies_merge[["Movie Title", "Year", "Studio","Total Gross","Theaters", "Opening", "Open", "Close"]]



In [7]:
#Add the Additional Columns
movies_df["Rated"] = ""
movies_df["Runtime"]= ""
movies_df["Director"]=""
movies_df["Actor 1"] = ""
movies_df["Actor 2"] = ""
movies_df["Rotten Tomatoes"] = ""
movies_df["Metacritic"] = ""
movies_df["IMDB Rating"] = ""
movies_df["Production Studio"] =""




In [8]:
for index, row in movies_df.iterrows():
    base_url = "http://www.omdbapi.com/?apikey=trilogy"
    params = {
        "t": row["Movie Title"],
        "y": row["Year"]
    }

    # make request and print url
    response = requests.get(base_url, params=params).json()
    
    
    try:
        movies_df.set_value(index, "Rated", response["Rated"])
        movies_df.set_value(index, "Runtime", response["Runtime"])
        movies_df.set_value(index, "Director", response["Director"])
        movies_df.set_value(index, "Actor 1", response["Actors"])
        movies_df.set_value(index, "Actor 2", response["Actors"])
        movies_df.set_value(index, "Rotten Tomatoes", response["Ratings"][1]["Value"])
        movies_df.set_value(index, "Metacritic", response["Ratings"][2]["Value"])
        movies_df.set_value(index, "IMDB Rating", response["Ratings"][0]["Value"])
        movies_df.set_value(index, "Production Studio", response["Production"])
            
        
    except (KeyError, IndexError):
        print(f" {movie} Missing field/result... skipping.")
    
   
    
    

In [9]:
#Clean data take out $ and commas to convert to integer
movies_df['Total Gross'] = movies_df['Total Gross'].str.replace('$','')
movies_df['Total Gross'] = movies_df['Total Gross'].str.replace(',','')
movies_df['Total Gross'] = pd.to_numeric(movies_df['Total Gross'])

movies_df['Opening'] = movies_df['Opening'].str.replace('$','')
movies_df['Opening'] = movies_df['Opening'].str.replace(',','')
movies_df['Opening'] = pd.to_numeric(movies_df['Opening'])

movies_df['Runtime'] = movies_df['Runtime'].str.replace('min','')
movies_df['Runtime'] = pd.to_numeric(movies_df['Runtime'])
movies_df = movies_df.rename(columns = {"Runtime":"Runtime (Minutes)"})

movies_df['Rotten Tomatoes'] = movies_df['Rotten Tomatoes'].str.replace('%','')
movies_df['Rotten Tomatoes'] = pd.to_numeric(movies_df['Rotten Tomatoes'])
movies_df = movies_df.rename(columns = {"Rotten Tomatoes":"Rotten Tomatoes (%)"})


In [10]:
for index, row in movies_df.iterrows():
    integer1 = int(row['Metacritic'].split('/')[0])
    integer2 = int(row['Metacritic'].split('/')[1])
    movies_df.set_value(index, "Metacritic", (integer1/integer2)*100)
    movies_df.set_value(index, "IMDB Rating", (float(row['IMDB Rating'].split('/')[0]))*10)
    movies_df.set_value(index, "Actor 1", str(row["Actor 1"].split(",")[0]))
    movies_df.set_value(index, "Actor 2", str(row["Actor 2"].split(",")[1]))
    movies_df.set_value(index, "Director", str(row["Director"].split(",")[0]))
  



In [11]:
movies_df.to_csv("output/movies_data.csv", encoding = "utf-8")